In [1]:
import requests
from zipfile import ZipFile
import pandas as pd
import polars as pl
import numpy as np
import pyarrow as pa

In [1]:
"""
    Read the es-en.txt, this is a database English-Spanish from  https://europat.net
    the file has a huge size, so we will use polars to read it.    
"""

'\n    Read the es-en.txt, this is a database English-Spanish from  https://europat.net\n    the file has a huge size, so we will use polars to read it.    \n'

In [2]:
df_euro_pat = pl.read_csv('es-en.txt', sep='\t', n_rows=4_000_000)

In [3]:
df_euro_pat.columns = ['spa', 'eng']

In [14]:
df_euro_pat.describe()

describe,spa,eng
str,str,str
"""count""","""4000000""","""4000000"""
"""null_count""","""0""","""0"""
"""mean""",null,null
"""std""",null,null
"""min""","""!n a typical b...","""! (XD CH3020-C..."
"""max""","""〉N-R³ or one o...","""❖ La expresión..."
"""median""",null,null


In [23]:
df_euro_pat.write_parquet('es-en.parquet')

In [19]:
# move eng->spa and spa->eng
df_euro_pat = df_euro_pat.with_columns([pl.col('spa').alias('eng'), pl.col('eng').alias('spa')])
df_euro_pat.head()

spa,eng
str,str
"""El contenido d...","""The styrene co..."
"""Dispositivo de...","""Adaptable supp..."
"""1, un sistema ...","""1, a communica..."
"""La inmunotrans...","""Immunoblotting..."
"""Análisis estad...","""Statistical an..."


In [21]:
df_euro_pat.write_csv('es-en_1.txt', separator='\t')

In [19]:
file_name = "spa-eng.zip"

url = f'https://www.manythings.org/anki/{file_name}'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

r = requests.get(url, headers=headers)
# save the file:
with open(file_name, 'wb') as file:
    file.write(r.content)
    
with ZipFile(file_name) as zip_file:
    zip_file.extractall("./spa-eng")

df_many_things = pd.read_csv('spa-eng/spa.txt', sep='\t', header=None)
df_many_things.columns = ['eng', 'spa', 'ignore']
df_many_things = df_many_things[['eng', 'spa']]
df_many_things.to_csv('spa-eng/spa.txt', sep='\t', index=False)

In [7]:
df_many_things = pd.read_csv('spa-eng/spa.txt', sep='\t')

In [9]:
df_many_things.describe()

,eng,spa
count,139636,139636
unique,118570,131221
top,You can put it there.,Estoy quebrado.
freq,68,12


In [18]:
# merge  datasets 
df_many_things_polars = pl.from_pandas(df_many_things)
df_many_things_polars.head()

eng,spa
str,str
"""Go.""","""Ve."""
"""Go.""","""Vete."""
"""Go.""","""Vaya."""
"""Go.""","""Váyase."""
"""Hi.""","""Hola."""


In [20]:
df_general = pl.concat([df_euro_pat, df_many_things_polars], rechunk=True)

ShapeError: unable to vstack, column names don't match: "spa" and "eng"